In [1]:
import pandas as pd
import math

In [2]:
# 电影评分集
data_path = "F:\\tmp\\u.data"

In [3]:
# DataFrame
udata = pd.read_csv(data_path, sep = "\t", header = None, names = ['user_id', 'item_id', 'rating', 'timestamp'])
udata.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
t = udata.groupby('item_id')['user_id'].count().reset_index()
t.columns = ['item_id', 'user_cnt']
t

,item_id,user_cnt
0,1,452
1,2,131
2,3,90
3,4,209
4,5,86
5,6,26
6,7,392
7,8,219
8,9,299
9,10,89


In [5]:
t.loc[t['item_id'] == 300, :]

,item_id,user_cnt
299,300,431


#### 构建数据格式 {user_id: {item_id: rating}}

In [6]:
train = dict()
for index, row in udata.iterrows():
    user_id = str(row['user_id'])
    item_id = str(row['item_id'])
    rating = row['rating']
    if train.get(user_id, -1) == -1:
        train[user_id] = dict()
    train[user_id][item_id] = rating
print(train)

{'196': {'242': 3, '393': 4, '381': 4, '251': 3, '655': 5, '67': 5, '306': 4, '238': 4, '663': 5, '111': 4, '580': 2, '25': 4, '286': 5, '94': 3, '692': 5, '8': 5, '428': 4, '1118': 4, '70': 3, '66': 3, '257': 2, '108': 4, '202': 3, '340': 3, '287': 3, '116': 3, '382': 4, '285': 5, '1241': 3, '1007': 4, '411': 4, '153': 5, '13': 2, '762': 3, '173': 2, '1022': 4, '845': 4, '269': 3, '110': 1}, '186': {'302': 3, '566': 5, '250': 1, '148': 4, '263': 3, '470': 5, '983': 3, '281': 4, '385': 4, '588': 4, '406': 1, '925': 5, '977': 3, '322': 5, '53': 1, '333': 3, '591': 4, '742': 3, '770': 2, '550': 4, '237': 2, '1277': 4, '1253': 4, '71': 5, '554': 1, '257': 4, '44': 5, '117': 5, '327': 3, '288': 1, '225': 4, '988': 4, '31': 4, '939': 5, '546': 4, '100': 4, '338': 3, '717': 3, '118': 2, '226': 5, '300': 5, '299': 3, '596': 4, '95': 3, '243': 2, '1016': 5, '79': 5, '306': 4, '106': 2, '829': 4, '934': 3, '1399': 2, '754': 2, '595': 3, '121': 2, '568': 4, '303': 3, '332': 4, '540': 4, '1046': 

#### 1、计算item1和item2相同的user的数量

In [7]:
# 两个item相同的user个数  分子
C = dict()
# item的用户数  分母
N = dict()

In [8]:
for u, items in train.items():
    for i in items:
        if N.get(i) == None:
            N[i] = 0
        N[i] += 1
        if C.get(i, -1) == -1:
            C[i] = dict()
        for j in items:
            if i == j:
                continue
            elif C[i].get(j, -1) == -1:
                C[i][j] = 0
            C[i][j] += 1

In [9]:
# 物品id302和504的相同用户
C['302']['504']

42

#### 2、计算相似度

In [10]:
W = dict()
for i, related_items in C.items():
    if W.get(i) == None:
        W[i] = dict()
    for j, cij in related_items.items():
        if W[i].get(j) == None:
            W[i][j] = 0
        # 相同用户 / 各自用户数的乘积再开方
        # jarcard 相似度
        W[i][j] = cij / math.sqrt(N[i] * N[j])

In [11]:
W['302'].__class__

dict

In [12]:
W['302'].items().__class__

dict_items

In [13]:
# 以列表返回可遍历的(键, 值) 元组数组
W['302'].items()

dict_items([('566', 0.24721232295804452), ('250', 0.2935262937838052), ('148', 0.2102812970896934), ('263', 0.13312051063847863), ('470', 0.20659110646994916), ('983', 0.10487553416094877), ('281', 0.22090379615265), ('385', 0.24944901728572158), ('588', 0.21229972629291133), ('406', 0.11605177063713189), ('925', 0.11941628680530643), ('977', 0.132630595013865), ('322', 0.369420832047477), ('53', 0.2513117940828043), ('333', 0.5164211580750208), ('591', 0.3174933507466257), ('742', 0.26278336186357715), ('770', 0.25362863675089403), ('550', 0.2266598197546936), ('237', 0.3346066798358886), ('1277', 0.10649640851078289), ('1253', 0.10965861582662817), ('71', 0.20342942828564212), ('554', 0.20108965722028524), ('257', 0.316682501187599), ('44', 0.24155161959602633), ('117', 0.3223291856101521), ('327', 0.48249790963716394), ('288', 0.49365185266738587), ('225', 0.13894679545337643), ('988', 0.1689414875501125), ('31', 0.22911709539825767), ('939', 0.19476884915604772), ('546', 0.28398769

In [14]:
import operator
# item_id为302的相似物品推荐top20
sorted(W['302'].items(), key = operator.itemgetter(1), reverse = True)[:20]

[('269', 0.5754123333067998),
 ('286', 0.5608995583955425),
 ('258', 0.5401099511231545),
 ('313', 0.5210711578231437),
 ('333', 0.5164211580750208),
 ('300', 0.5086915555366447),
 ('340', 0.4980499267495288),
 ('301', 0.4973947644973536),
 ('303', 0.4962546289118298),
 ('288', 0.49365185266738587),
 ('268', 0.4869187332359757),
 ('327', 0.48249790963716394),
 ('347', 0.48087613683680047),
 ('272', 0.478351532260688),
 ('315', 0.4770842982214229),
 ('328', 0.4729756721205128),
 ('346', 0.46007274847479945),
 ('307', 0.4528210718727978),
 ('245', 0.44946657497549475),
 ('270', 0.44781107551989907)]